<center><font color="green" size="6"> <b>Sugarcane Yield Prediction using RS and ML<br /><b> </font></center>
<center><img src="https://www.omex.com/wp-content/uploads/2020/10/Sugar-Cane-Harvest-1536x772.jpg" height="150"></center>

---

_Script elaborated by **Raul Roberto Poppiel**_ ([raulpoppiel@gmail.com](raulpoppiel@gmail.com)) for the [FAPESP](https://fapesp.br/en) project Nº [23/01062-1](https://bv.fapesp.br/en/bolsas/207973/satellite-imagery-and-machine-learning-for-sugarcane-yield-estimation-in-regions-of-sao-paulo-state/), led by Professor Ana Claudia dos Santos Luciano (analuciano@usp.br) from ESALQ/USP, Brazil. The methodology employed was developed by Rafaella Pironato Amaro (rafaellapironato.amaro@gmail.com) and is detailed in the document titled [Estimativa de produtividade da cana-de-açúcar a partir de imagens do satélite Sentinel-2A e o algoritmo de aprendizagem de máquina Random Forest](https://doi.org/10.11606/D.11.2023.tde-02102023-163947).

### Install and import tools

Geospatial modules

In [ ]:
# Load modules
import ee
import geemap

print('Modules loaded')

Modules loaded


Data science modules

In [ ]:
# Load modules
import os
import pandas as pd
import numpy as np
import time
from pathlib import Path

print('Modules loaded')

Modules loaded


### Connect to GEE and GDrive

In [ ]:
# GEE Authentication
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1AfQc5k6i48PKbF1kxiWari0k9nzAG7LBbVzvXuYhmQ&tc=vU5FeXNrvBi4y5riifNwdd2EDEJx30fwbI1AtIc_EGw&cc=AtaooITMVrJDgKZPfNLY8ikPAppC0TiG8a_Wo8KoEZA

The authorization workflow will generate a code, which you should paste in the box below.


In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set GDrive paths

Define the output folders in GD

In [ ]:
# Define folder names
folder_name_root = 'Colab Notebooks'
folder_name_project = '01_SugarcaneYieldPrediction'
folder_name_farm = 'usinas_all'
folder_name_specific = '02_climatic_data_daily'

# define your GEE username
username_gee = 'raulrpoppiel'

# Define sugarcane crop-season (SAFRA)
CropSeason = 1920 # 1920, 2021, 2122, 2223

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}'
project_path = f'{root_path}/{folder_name_project}'
farm_path = f'{project_path}/{folder_name_farm}'
out_path = f'{farm_path}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully.")
else:
  print("Output directory already exists.")

os.chdir(out_path)
print(os.getcwd(),'\n')  # Print the current working directory
pd.DataFrame(os.listdir(), columns=['List files'])  # List files and directories in the current directory

Output directory already exists.
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily 

List of files:
                                                Files
0  02_climatic_data_daily_MEAN_safra_1920_merged.csv
1  02_climatic_data_daily_MEAN_safra_2021_merged.csv
2  02_climatic_data_daily_MEAN_safra_2122_merged.csv


## Project-specific settings

### Feature collection

Example of attributes table structure:
```
 'ID_SIG': 'X19200890000003600027002', 🟢
 'AREA': 46.0963242244,
 'BLOCO': 2,
 'CODFAZ': 36,
 'COD_USINA': 89,
 'EST_Corte': 1, 🟢
 'Local': 'Usina 3',
 'Ordem': 'Latossolos',
 'POL': 0,
 'SAFRA': '1920', 🟢
 'SAFRA_real': 1819, 🟢
 'TAH': 0,
 'TALHAO': 17002,
 'TCH_ANT': 88.875, 🟢
 'Unidade_So': 'LV21',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',🟢
 'relevo': 'Suave Ondulado',
 'soloGeral': 'LV',
 'usina': 'M3'
```

Import the features from your Assets

In [ ]:
# Import polygons of agricultural plots (shapefiles) from your GEE

fc1 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_1')
# fc2 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_2')
# fc3 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_3')
# fc4 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_4')
# fc5 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_5')
# fc6 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_6')
# fc7 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_7')
# fc8 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_8')
# fc9 =  ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_9')
# fc10 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_10')
# fc11 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_11')
# fc12 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_12')
# fc13 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_13')
# fc14 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_14')
# fc15 = ee.FeatureCollection(f'users/{username_gee}/Sugarcane/Safra_{CropSeason}_15')

print('FC1 size: ',fc1.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('FC1 attribute table (1st feature): \n')
display(fc1.limit(1).getInfo()['features'][0]['properties'])

FC1 size:  321 

FC1 attribute table (1st feature): 



{'AREA': 2.43708017691,
 'BLOCO': '0',
 'CODFAZ': '1',
 'COD_USINA': '28',
 'EST_Corte': '3',
 'ID_SIG': 'X22230280001000100000030',
 'Local': 'Usina 1',
 'Ordem': 'Nitossolos',
 'POL': '0',
 'SAFRA': 2223,
 'SAFRA_real': '2122',
 'TAH': '0',
 'TALHAO': '30',
 'TCH_ANT': '112.79',
 'Unidade_So': 'NV7',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',
 'relevo': 'Ondulado',
 'soloGeral': 'NV',
 'usina': 'M1'}

In [ ]:
# Select an attribute name (column) that contains the ID of polygons
ID = 'ID_SIG'

fc_to_reduction1 = fc1.select(ID)
# fc_to_reduction2 = fc2.select(ID)
# fc_to_reduction3 = fc3.select(ID)
# fc_to_reduction4 = fc4.select(ID)
# fc_to_reduction5 = fc5.select(ID)
# fc_to_reduction6 = fc6.select(ID)
# fc_to_reduction7 = fc7.select(ID)
# fc_to_reduction8 = fc8.select(ID)
# fc_to_reduction9 = fc9.select(ID)
# fc_to_reduction10 = fc10.select(ID)
# fc_to_reduction11 = fc11.select(ID)
# fc_to_reduction12 = fc12.select(ID)
# fc_to_reduction13 = fc13.select(ID)
# fc_to_reduction14 = fc14.select(ID)
# fc_to_reduction15 = fc15.select(ID)

In [ ]:
# Make a colection of collecions of geometries
SugarcaneData = ee.FeatureCollection([fc1
                                      # ,fc2,fc3,fc4,fc5,fc6,fc7,fc8,fc9,fc10,fc11,fc12,fc13,fc14,fc15
                                      ]).flatten()

# Display the number os polygons
print('FC size: ',SugarcaneData.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('Attribute table structure (1st feature): \n')
display(SugarcaneData.limit(1).getInfo()['features'][0]['properties'])

FC size:  4815 

Attribute table structure (1st feature): 



{'AREA': 2.43708017691,
 'BLOCO': '0',
 'CODFAZ': '1',
 'COD_USINA': '28',
 'EST_Corte': '3',
 'ID_SIG': 'X22230280001000100000030',
 'Local': 'Usina 1',
 'Ordem': 'Nitossolos',
 'POL': '0',
 'SAFRA': 2223,
 'SAFRA_real': '2122',
 'TAH': '0',
 'TALHAO': '30',
 'TCH_ANT': '112.79',
 'Unidade_So': 'NV7',
 'VAR': 'CTC4',
 'VARIEDADE': 'CTC4',
 'relevo': 'Ondulado',
 'soloGeral': 'NV',
 'usina': 'M1'}

### Crop-season and real period

In [ ]:
# Define the real period in years
if CropSeason == 1920:
    SrtYear = '2018'
    EndYear = '2019'
elif CropSeason == 2021:
    SrtYear = '2019'
    EndYear = '2020'
elif CropSeason == 2122:
    SrtYear = '2020'
    EndYear = '2021'
elif CropSeason == 2223:
    SrtYear = '2021'
    EndYear = '2022'
elif CropSeason == 2324:
    SrtYear = '2022'
    EndYear = '2023'
else:
    # Handle unknown CropSeason values
    print("Unknown CropSeason")

# Example:
# SAFRA_real = SAFRA - 1
SrtYear_real = str(int(SrtYear) - 1)
EndYear_real = str(int(EndYear) - 1)

# Display the results
print(f"Start: 01/04/{SrtYear} --> Real: 01/04/{SrtYear_real}")
print(f"End: 31/03/{EndYear} --> Real: 31/03/{EndYear_real}")

Start: 01/04/2021 --> Real: 01/04/2020
End: 31/03/2022 --> Real: 31/03/2021


In [ ]:
# Use real years for RS data acquisition (DO NOT change it)

# Daily (for Hydric Balance calculations)
SrtDate_daily = SrtYear+"-01-01" # include the first 3 months before crop-season for BH calculations
EndDate_daily = EndYear+"-03-31"

### Filter Sugarcane field data and bounding box

In [ ]:
# Compute bounding box for the fc
buffer_size = 5000; # define a value in meters

bbox_rect = SugarcaneData.geometry().bounds().buffer(buffer_size).bounds().getInfo()
bbox_coords = bbox_rect.get('coordinates')[0]
bbox = ee.Geometry.Rectangle([bbox_coords[0][0],bbox_coords[0][1],bbox_coords[2][0],bbox_coords[2][1]],None,False)

Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

Map.addLayer(SugarcaneData,{'color': 'FF0000'},'fc')
Map.addLayer(bbox, {'color': '#000060'}, 'Bbox')
Map.centerObject(SugarcaneData, 8)

Map.setControlVisibility()
Map

Map(center=[-21.242562902824027, -50.06488035654671], controls=(WidgetControl(options=['position', 'transparen…

## Daily climatic data

#### Import data: T, P, R

Here we get data from the ECMWF ERA-5 land-daily:
* air temperature, in °C = [K - 273.15]
* precipitation, in mm = [m * 1000]
* radiation, in MJ/m^2 = [J/m^2 * 0.000001]
* https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_DAILY_AGGR

Helper functions

In [ ]:
# Function to convert unit and replace the original band
def convert_units_daily(image):
    img1 = image.select('temperature_2m').subtract(273.15)
    img2 = image.select('total_precipitation_sum').multiply(1000)
    img3 = image.select('surface_solar_radiation_downwards_sum').multiply(0.000001);

    img4 = image.select('surface_net_thermal_radiation_sum').multiply(0.000001);
    img5 = image.select('surface_net_solar_radiation_sum').multiply(0.000001);

    col = ee.Image.cat(img1,img2,img3,img4,img5)
    return col


# Function to calculate Potential Evapotranspiration (Priestley-Taylor)
def calc_etp(image):
    # Temperature in °C
    tp = image.select('temperature_2m')

    # Rule 1: 0 < tp < 16
    w1 = tp.multiply(0.0145).add(0.407)

    # Rule 2: 16 =< tp < 32
    w2 = tp.multiply(0.01).add(0.483)

    # Additional rule: 32 =< tp
    w3 = tp.multiply(1)  # Define your custom equation here

    # Combine the results based on conditions
    w = (ee.Image(0).where(tp.gt(0).And(tp.lt(16)), w1)
                   .where(tp.gte(16).And(tp.lt(32)), w2)
                   .where(tp.gte(32), w3)
                   .rename('w'))

    # Total net radiation
    Rn = image.select('surface_net_thermal_radiation_sum').add(image.select('surface_net_solar_radiation_sum'))

    # Potential Evapotranspiration
    etp1 = w.multiply(Rn).multiply(1.26)
    etp = etp1.divide(2.45).rename('ETP')

    return image.addBands(etp)

In [ ]:
# Import collection
daily_tp_etp = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
                .filterBounds(bbox)
                .filterDate(SrtDate_daily, EndDate_daily)
                .map(convert_units_daily)
                .map(calc_etp)
                .select(['temperature_2m','total_precipitation_sum','ETP']))

In [ ]:
# Select and display one of the images

# Get the most recent image
clmostRecent = daily_tp_etp.first().clip(bbox)
print('Date:',clmostRecent.get('system:index').getInfo())

print('\n Band Names:', clmostRecent.bandNames().getInfo())

# Get values from image
reducers = ee.Reducer.percentile([1,98], ['min','max'])
stats = clmostRecent.select(['ETP.*']).rename('x').reduceRegion(reducer = reducers,
                              geometry = bbox,
                              scale = 5000,
                              tileScale = 16)

img_min = ee.Number(stats.get('x_min').getInfo()).format('%.2f').getInfo();
img_max = ee.Number(stats.get('x_max').getInfo()).format('%.2f').getInfo();
print('\n','PercMin: '+str(img_min)+' (mm day−1)',' |  PercMax: '+str(img_max)+' (mm day−1)')

# Display the image
Map = geemap.Map(basemap='Esri.WorldImagery')
Map.setOptions()

etp_palette = {#'bands': ['ETP.*'],
                 'palette': ['d7191c', 'fdae61', 'ffffbf', 'abd9e9', '2c7bb6'],
                 'min': [stats.get('x_min')],
                 'max': [stats.get('x_max')]
                 }

Map.addLayer(clmostRecent.select('ETP.*'), etp_palette,'Most Recent Image - ETP')
Map.centerObject(SugarcaneData, 8)

Map.setControlVisibility()
Map

Date: 20210101

 Band Names: ['temperature_2m', 'total_precipitation_sum', 'ETP']

 PercMin: 1.66 (mm day−1)  |  PercMax: 4.79 (mm day−1)


Map(center=[-21.242562902824027, -50.06488035654671], controls=(WidgetControl(options=['position', 'transparen…

### Reduce by region and export as tables

In [ ]:
# Define the statistic
# "MEAN", "MAXIMUM", "MEDIAN","MINIMUM","MODE","STD","MIN_MAX","SUM","VARIANCE", "COUNT"
statistics_Type = 'MEAN'

# Define the output path to save results
global_stats_path1 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_1.csv')
# global_stats_path2 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_2.csv')
# global_stats_path3 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_3.csv')
# global_stats_path4 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_4.csv')
# global_stats_path5 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_5.csv')
# global_stats_path6 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_6.csv')
# global_stats_path7 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_7.csv')
# global_stats_path8 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_8.csv')
# global_stats_path9 =  os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_9.csv')
# global_stats_path10 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_10.csv')
# global_stats_path11 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_11.csv')
# global_stats_path12 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_12.csv')
# global_stats_path13 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_13.csv')
# global_stats_path14 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_14.csv')
# global_stats_path15 = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_15.csv')

print(global_stats_path1)
# print(global_stats_path2)
# print(global_stats_path3)
# print(global_stats_path4)
# print(global_stats_path5)
# print(global_stats_path6)
# print(global_stats_path7)
# print(global_stats_path8)
# print(global_stats_path9)
# print(global_stats_path10)
# print(global_stats_path11)
# print(global_stats_path12)
# print(global_stats_path13)
# print(global_stats_path14)
# print(global_stats_path15)

/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_1.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_2.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_3.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_4.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_5.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_6.csv
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02

In [ ]:
# Reduzing daily climatic values by zone
scale = 100.0

parameters = {
    'in_value_raster': daily_tp_etp,
    'statistics_type':statistics_Type,
    'scale':float(scale), # a high scale avoid missing values
    'tile_scale':16.0,
    'return_fc':False,
    'timeout':3000
    }

In [ ]:
geemap.zonal_statistics(in_zone_vector=fc_to_reduction1,out_file_path=global_stats_path1,**parameters)
time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction2,out_file_path=global_stats_path2,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction3,out_file_path=global_stats_path3,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_1.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_2.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_3.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction4,out_file_path=global_stats_path4,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction5,out_file_path=global_stats_path5,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction6,out_file_path=global_stats_path6,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_4.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_5.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_6.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction7,out_file_path=global_stats_path7,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction8,out_file_path=global_stats_path8,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_7.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_8.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction9,out_file_path=global_stats_path9,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction10,out_file_path=global_stats_path10,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_9.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_10.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction11,out_file_path=global_stats_path11,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction12,out_file_path=global_stats_path12,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction13,out_file_path=global_stats_path13,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_11.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_12.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_13.csv


In [ ]:
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction14,out_file_path=global_stats_path14,**parameters)
# time.sleep(2)
# geemap.zonal_statistics(in_zone_vector=fc_to_reduction15,out_file_path=global_stats_path15,**parameters)
# time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_14.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_15.csv


### Merge tables into a single one

In [ ]:
# Define the paths to your CSV files
file_paths = [
    os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_{i}.csv')
    for i in range(1, 16)
]

print(len(file_paths), 'paths')

15 paths


In [ ]:
# Create an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through the CSV files and merge them
for file_path in file_paths:
    if os.path.exists(file_path):
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Remove the "system:index" column if it exists
        if "system:index" in df.columns:
            df = df.drop(columns=["system:index"])

        # Merge the data into the merged_df
        if merged_df.empty:
            merged_df = df
        else:  # Fix the typo here (remove "ID")
            merged_df = pd.concat([merged_df, df], axis=0, ignore_index=True)

# Move the last column (ID_SIG) to the first position
merged_df = merged_df[[merged_df.columns[-1]] + list(merged_df.columns[:-1])]
print('Dataframes merged!')

Dataframes merged!


In [ ]:
# Define the path for the output merged CSV file
merged_csv_path = os.path.join(out_path, f'02_climatic_data_daily_{statistics_Type}_safra_{CropSeason}_merged.csv')

# Save the merged DataFrame to a CSV file
merged_df.to_csv(merged_csv_path, encoding='utf-8', index=False)

# Print the path to the merged CSV file
print(f'Merged CSV file saved at: {merged_csv_path}')

Merged CSV file saved at: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_merged.csv


In [ ]:
# Delete CSV files single that were merged
for file_path in file_paths:
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_1.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_2.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_3.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_4.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_5.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/02_climatic_data_daily/02_climatic_data_daily_MEAN_safra_2223_6.csv
Deleted: /content/drive/MyDrive/Colab Notebooks/01_S